# LiveData - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu `LiveData` - części architektury **MVVM**.

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMDExYnllNDZwZW40NjAxbzl4ZTNlZXN0dHZ1aXE5MjFha2g2ZXV1ayZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xYi3Uxdr0LCyxmoDJO/giphy.gif" width="200" />

`LiveData` jest częścią bibliotek Androida i jest obiektem, który przechowuje dane **obserwowane** przez komponenty aplikacji, takie jak aktywności, fragmenty czy `ViewModel`. Jest zaprojektowany tak, aby dostarczać reaktywne i cykliczne powiadomienia o zmianach danych.

**Reaktywność** to podejście w programowaniu, które koncentruje się na tym, aby system reagował na zmiany i propagował te zmiany w sposób automatyczny. W kontekście aplikacji, reaktywność odnosi się do zdolności systemu do dynamicznego reagowania na zmiany danych i propagowania tych zmian do odpowiednich komponentów. 

Opiera się na obsłudze zdarzeń, które są generowane w systemie w wyniku **zmian w danych**. Zdarzenia te mogą być przekształcane i łączone za pomocą różnych operacji, tworząc **strumienie danych**. Strumienie są sekwencją wartości, które mogą być emitowane i subskrybowane (obserwowane) przez komponenty. Automatycznie propaguje zmiany w danych do komponentów, które subskrybują te dane. Oznacza to, że komponenty **nie muszą** ręcznie monitorować i aktualizować danych, ponieważ system sam zarządza tym procesem.

- Obserwowanie zmian - Komponenty mogą subskrybować (obserwować) `LiveData`, aby otrzymywać powiadomienia o zmianach danych. Kiedy wartość zostanie zaktualizowana, subskrybenci (obserwatorzy) otrzymają aktualizację i będą mogli zareagować na nową wartość.
- Bezpieczeństwo wątków - `LiveData` automatycznie zarządza wątkami, zapewnia, otrzymanie powiadomień na wątku `UI` (**głównym wątku**). Jeśli wartość `LiveData` zostanie zmieniona w wątku innym niż wątek główny, powiadomienia zostaną dostarczone do obserwatorów w wątku głównym, co ułatwia aktualizację interfejsu użytkownika.
- Unikanie wycieków pamięci: `LiveData` jest **świadome cyklu życia komponentów** i automatycznie **zatrzymuje obserwację**, gdy komponenty, które go subskrybują, są w **stanie nieaktywnym**, takim jak aktywność w stanie pauzy lub fragment w stanie `detach`. Dzięki temu unika się wycieków pamięci, które mogą wystąpić, gdy komponenty trzymają referencje do innych komponentów poza ich cyklem życia.

`LiveData` jest użyteczne w przypadku komunikacji między **warstwami aplikacji**, takimi jak `ViewModel` i interfejs użytkownika, ponieważ umożliwia bezpośrednie powiązanie danych z widokami (elementami ui), eliminując potrzebę ręcznej synchronizacji danych.

`LiveData` jest bardziej odpowiednie do **tradycyjnych** (gdzie ui tworzony jest w oparciu o `View` z `Fragment`) aplikacji Android. `State` natomiast jest dedykowane dla aplikacji opartych na **Jetpack Compose**, które używają **deklaratywnego podejścia** do budowania `UI`. Oba mechanizmy dostarczają reaktywności i automatycznego odświeżania widoku w odpowiedzi na zmiany danych.

Utworzymy prosty licznik, w którym wartość licznika będzie obserwowana. komponent odpowiedzialny za renderowanie ekranu będzie reagował automatycznie na zmiany tej wartości.

Aby skorzystać z `LiveData` i `ViewModel` musimy dodać odpowiednie zależności.

In [ ]:
implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'

Rozpocznijmy od utworzenia `ViewModel`

### ViewModel

In [ ]:
public class CounterViewModel extends ViewModel {
    private final MutableLiveData<Integer> counter = new MutableLiveData<>(0);

    public LiveData<Integer> getCounter() {
        return counter;
    }

    public void increase(){
        if (counter.getValue() != null)
            counter.setValue(counter.getValue() + 1);
    }

    public void decrease(){
        if (counter.getValue() != null)
            counter.setValue(counter.getValue() - 1);
    }

    public void clear(){
        if (counter.getValue() != null)
            counter.setValue(0);
    }
}

- `private final MutableLiveData<Integer> counter = new MutableLiveData<>(0);` - `MutableLiveData` jest klasą służącą do przechowywania i zarządzania danymi reaktywnymi. pole przechowuje aktualną wartość licznika i jest zainicjalizowane wartością 0. Jeżeli chcemy zmienić wartość z poziomu `CounterFragment`, wywołujemy odpowiednią metodę dostępową, która odpowiednio modyfikuje to pole.
- `public LiveData<Integer> getCounter()` Getter dostarcza dostęp do pola `counter` tylko do odczytu, umożliwiając obserwowanie jego wartości bez możliwości zmiany.
- `public void increase()` Metoda dostępowa zwiększająca stan licznika.
- `public void decrease()` Metoda dostępowa zmniejszająca stan licznika.
- `public void clear()` Metoda dostępowa rresetująca stan licznika.

Klasa `CounterViewModel` dostarcza interfejs do zarządzania stanem licznika poprzez metody `increase()`, `decrease()` i `clear()`. Pole `counter` umożliwia obserwowanie wartości licznika i reagowanie na jego zmiany w innych komponentach są subskrybentami (blokuje możliwość bezpośredniej zmiany licznika - do tego służą metody dostępowe).

### CounterFragment

Rozpocznijmy od zdefiniowania layoutów dla aktywności głównej oraz fragmentu. Podobnie jak we wcześniejszej aplikacji, wykorzystujemy jeden fragment, więc nie korzystamy z nawigacji.

In [ ]:
// fragment_counter.xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <TextView
        android:id="@+id/show_count"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:gravity="center"
        android:text="0"
        android:textSize="250sp"
        android:textStyle="bold" />

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <Button
            android:id="@+id/increaseButton"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:layout_weight=".5"
            android:text="Increase" />

        <Button
            android:id="@+id/decreaseButton"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:layout_weight=".5"
            android:text="Decrease" />
    </LinearLayout>

    <Button
        android:id="@+id/resetButtton"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="Reset" />
</LinearLayout>

In [ ]:
// activity_main.xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/fragment_main"
        android:name="com.example.livedatabasics_kotlin.ui.fragment.CounterFragment"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
</androidx.constraintlayout.widget.ConstraintLayout>

Dodajmy instancję `CounterViewModel` do fragmentu.

In [ ]:
private CounterViewModel viewModel;

W metodzie `onCreateView` zainicjujmy `viewModel` i dodajmy nasz fragment jako obserwatora do pola `counter` klasy `CounterViewModel`

In [ ]:
viewModel = new ViewModelProvider(requireActivity()).get(CounterViewModel.class);

viewModel.getCounter().observe(getViewLifecycleOwner(),
        newValue -> binding.showCount.setText(newValue.toString()));

Przy użyciu metody `observe()` na `LiveData` dodajemy fragment jako obserwatora (subskrybenta) pola `counter`. `viewLifecycleOwner` wskazuje, że obserwator ma być związany z cyklem życia widoku (dzięki temu `LiveData` *wie* kiedy wstrzymać aktualizacje - kiedy fragment znajduje się w stanie nieaktywnym). Kiedy wartość `counter` w `viewModel` ulegnie zmianie, przekaże nową wartość do lambdy `newValue -> ... `. W tym przypadku, nowa wartość `counter` jest przypisywana do `binding.showCount.text`.

W efekcie, za każdym razem, gdy wartość `counter` w `viewModel` ulegnie zmianie, tekst w `binding.showCount.text` zostanie zaktualizowany i widok będzie wyświetlał nową wartość.

Dodajmy obsługę przycisków.

In [ ]:
binding.increaseButton.setOnClickListener(view -> viewModel.increase());
binding.decreaseButton.setOnClickListener( view -> viewModel.decrease());
binding.resetButtton.setOnClickListener (view -> viewModel.clear());

Za każdym razem, gdy zmieniamy wartość `_counter` za pomocą metod dostępowych (`increase()`, `decrease()`, `clear()`), tekst w polu `binding.showCount.text` zostanie zaktualiowany.

Możemy przetestować aplikację

<table><tr><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMDExYnllNDZwZW40NjAxbzl4ZTNlZXN0dHZ1aXE5MjFha2g2ZXV1ayZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xYi3Uxdr0LCyxmoDJO/giphy.gif" width="150" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExZWl6emZ3cWxvYzByMDJ1aWdtbG94eTlyZzI0MTh4ejluejg1dnhzNiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/S3uYswjxuVK1ILsds0/giphy.gif" width="200" /></td></tr></table>